### Step1 导入相关包

In [1]:
import os

from langchain_community.llms import BaichuanLLM
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

### Step2 加载数据

In [2]:
data = "../demo/"
# 这里为了做演示，随便找了一个pdf

In [3]:
loader = PyPDFDirectoryLoader(data)

docs_before_split = loader.load()
# 过滤目录和附录
docs_before_split = [doc for doc in docs_before_split if doc.metadata['page'] > 7 and doc.metadata['page'] <275] 
# 这里可以理解为对自己的文档切分成块，chuck size是每一块的大小，可以根据需求调整
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 256,
    chunk_overlap  = 30,
)
docs_after_split = text_splitter.split_documents(docs_before_split)

docs_after_split[0]

Document(page_content='第一章 电力现货市场基础   \n             \n1   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现', metadata={'source': '..\\demo\\电力现货市场101问.pdf', 'page': 8})

### Step3 创建向量数据库

In [4]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openai embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 使用m3e模型做embeddding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

d:\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\miniconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

### Step4 创建QA链

In [50]:
# 创建检索器
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

In [56]:
query = """电力市场与普通商品市场的差异"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query,k=25)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")


# 从这一步看，已经很清晰rag的原理了，即从大量文件索引找到top k相关的text块，供下一步LLM查找、总结答案

检索到的第1个内容: 
 品种、交易时间、竞争模式等维度进一步细分，各分类市场的知识将在后续问题讨论中详述。  
（2）电力市场和普通商品市场的差异。  
电力市场相较于普通商品市场具有显著的特殊性， 归根到底就在于电力商品与其他一
般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社
会属性。  
电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远
快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。
-----------------------------------------------------
检索到的第2个内容: 
 电力现货市场 101 问  
  42 
 
图1 − 4 现货市场时间轴  
（1）集中式市场模式。  
市场供需双方在每天特定时间之前向调度机构报价， 由调度机构根据供需双方报价和
网络条件等出清。在不同国家，日前市场的名称有所不同。例如，在挪威和美国 PJM电
力市场称日前（ day-ahead ）交易，在澳大利亚称短期提前（ short-run ahead ）交易，在我
-----------------------------------------------------
检索到的第3个内容: 
 电力商品具有同质性。电能不带有任何生产者的标识，电能生产者将生产的电能输入
-----------------------------------------------------
检索到的第4个内容: 
 电力现货市场 101 问  
  30 和一个双边市场，给予了市场充分的流动性，也为市场成员提供了多次报价的机会。  
4）美国纽约州容量市场。  
美国纽约州 NYISO运营着一个装机容量（ install capacity ，ICAP）市场。该市场的关
键特征是它是一个短期市场，虽然装机容量市场的目的是确保长期的资源充裕性，但不
提供长期的价格信号，也不锁定未来几年的特定容量价格。  
NYISO的ICAP市场有两个为期 6个月的容量期， 即冬季和夏季容量期。 通过三次拍
-----------------------------------------------------
检索到的第5个内容: 
 市场”的概念，即在狭义电力市场的基础上，将电力市场化改革领

In [61]:
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.callbacks import CallbackManagerForRetrieverRun

from typing import List
from sentence_transformers import CrossEncoder

""" 使用自定义的检索器,对上面相关性检索召回的文本块进行重排"""
class CustomRetriever(BaseRetriever):
    
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:
        cross_encoder = CrossEncoder("BAAI/bge-reranker-base", max_length=512, device="cpu")
        reranked_docs = cross_encoder.rank(
        query,
        [doc.page_content for doc in relevant_documents],
        top_k=5,
        return_documents=True,)
        reranking_relevant_documents= []
        for doc in reranked_docs:
           reranking_relevant_documents.append(Document(page_content=doc["text"],metadata=relevant_documents[doc["corpus_id"]].metadata)) 
        return reranking_relevant_documents

In [62]:
# 自定义加入重排的检索器
retriever = CustomRetriever()

In [63]:
# 创建prompt
prompt_template = """
请用下面相关文本回答问题，如果不知道答案，就回复不知道，

{context}

Question: {question}

Helpful Answer:
"""

prompt = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [64]:
# 创建百川的LLM
llm = BaichuanLLM()

In [65]:
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt}
)

### Step5 提问，运行QA链，得到RAG结果

In [66]:
question = "电力市场与普通商品市场的差异?"
result = qa_chain({"query": question})
# result["result"]
print("========= chain result ==========")
print(result)

d:\miniconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


========= chain result ==========
{'query': '电力市场与普通商品市场的差异?', 'result': '电力市场与普通商品市场的差异主要体现在电力商品的自然属性和社会属性上。电力商品具有无仓储性，其生产、交割和消费几乎是同时完成的，交割速度远快于一般商品，因此不存在一般商品一手交钱一手交货的交易方式。此外，电力商品具有同质性，电能不带有任何生产者的标识。由于电力商品的难以储存和快速性等特点，电力市场需要维持实时平衡的特殊的市场规则。', 'source_documents': [Document(page_content='品种、交易时间、竞争模式等维度进一步细分，各分类市场的知识将在后续问题讨论中详述。  \n（2）电力市场和普通商品市场的差异。  \n电力市场相较于普通商品市场具有显著的特殊性， 归根到底就在于电力商品与其他一\n般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社\n会属性。  \n电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远\n快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。', metadata={'source': '..\\demo\\电力现货市场101问.pdf', 'page': 9}), Document(page_content='交易规则对电力也适用，不需要任何特殊的交易规则和协议。其实不然，电力商品较之普通商品有其无可争议的特殊性。问题 1从商品属性回答了电力市场与普通商品市场的\n差异，要深入了解电力现货市场的特殊性，则必须深刻理解电力的自然属性，自然属性\n的电力特殊性是与电力现货市场交易规则的特殊性高度一致的。  \n1）电力具有难以储存的特殊性。这一特点导致供需双方提前达成的交易结果与需求\n方实时消费的电能量在数量上必然存在偏差，为弥补这种偏差，势必需要维持实时平衡\n的特殊的市场规则。', metadata={'source': '..\\demo\\电力现货市场101问.pdf', 'page': 20}), Document(page_content='电力商品具有同质性。电能不带有任何生产者的标识，电能生产者将生产的电能输入', metadata={'source': '..\\demo\\电力现

### Step6 开始评估

In [68]:
result['ground_truths'] = "电力市场相较于普通商品市场具有显著的特殊性，归根到底就在于电力商品与其他一般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社会属性。电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。电力商品具有同质性。电能不带有任何生产者的标识，电能生产者将生产的电能输入电网，即完成了生产过程；而电能的使用者也只能从电网获取所需数量的电能，电能生产者和消费者间可以达成交易，但在电能实际生产和消费过程中不存在对应性。电力商品具有可预测性。电能需求在较长周期内会以日或周为单位呈现周期性波动，一定程度上抑制了投机行为，但也增加了市场主体滥用市场力的可能性。电力商品具有生产资料和生活资料的双重属性，因此它既关系国计，又关系民生。电力市场既是生产资料市场，又是生活资料市场，还是十分典型的无仓储公共市场。"

from ragas.metrics import faithfulness, answer_relevancy, context_relevancy, context_recall
from ragas.langchain.evalchain import RagasEvaluatorChain

# make eval chains
eval_chains = {
    m.name: RagasEvaluatorChain(metric=m) 
    for m in [faithfulness, answer_relevancy, context_relevancy, context_recall]
}

# evaluate
for name, eval_chain in eval_chains.items():
    score_name = f"{name}_score"
    print(f"{score_name}: {eval_chain(result)[score_name]}")


faithfulness_score: 1.0
answer_relevancy_score: 0.8896998015854355
context_relevancy_score: 0.14705882352941177
context_recall_score: 1.0
